# Spam Detection Pipeline

Complete implementation with:
1. Data Preprocessing
2. Feature Extraction (Binary, Count, TF-IDF, Word2Vec)
3. Model Training & Evaluation (Naive Bayes, SVM, Random Forest)

## 1. Import Libraries

In [ ]:
import re
import string
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import os

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from gensim.models import Word2Vec

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

## 2. Load Dataset

In [ ]:
# Path to dataset
path = r"C:\Users\Sroor For Laptop\.cache\kagglehub\datasets\uciml\sms-spam-collection-dataset\versions\1"
filepath = os.path.join(path, "spam.csv")

# Load data
df = pd.read_csv(filepath, encoding='latin-1')

# Rename columns
df.rename(columns={'v1': 'label', 'v2': 'text'}, inplace=True)

# Keep only relevant columns
df = df[['label', 'text']]

print(f"Dataset shape: {df.shape}")
print(f"\nClass distribution:\n{df['label'].value_counts()}")
df.head()

## 3. Data Preprocessing

In [ ]:
# Initialize preprocessing tools
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    """Clean and preprocess text"""
    # Lowercase
    text = text.lower()
    
    # Remove special characters and numbers
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\w*\d\w*', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Tokenization
    tokens = word_tokenize(text)
    
    # Remove stopwords and lemmatize
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    
    return ' '.join(tokens)

# Apply preprocessing
print("Preprocessing text...")
df['cleaned_text'] = df['text'].apply(clean_text)
df['clean_tokens'] = df['cleaned_text'].apply(word_tokenize)

print("Preprocessing complete!")
df[['text', 'cleaned_text']].head()

## 4. Feature Extraction

### a) Binary Encoding (Sentence-level)

In [ ]:
print("Extracting Binary Encoding features...")
binary_vectorizer = CountVectorizer(binary=True)
X_binary = binary_vectorizer.fit_transform(df['cleaned_text'])
print(f"Binary Encoding Shape: {X_binary.shape}")

### b) Count Vectorization (Sentence-level)

In [ ]:
print("Extracting Count Vectorization features...")
count_vectorizer = CountVectorizer()
X_count = count_vectorizer.fit_transform(df['cleaned_text'])
print(f"Count Vectorization Shape: {X_count.shape}")

### c) TF-IDF (Sentence-level)

In [ ]:
print("Extracting TF-IDF features...")
tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(df['cleaned_text'])
print(f"TF-IDF Shape: {X_tfidf.shape}")

### d) Word2Vec (Word-level)

In [ ]:
print("Training Word2Vec model on words...")

# Prepare tokenized sentences
tokenized_sentences = df['clean_tokens'].tolist()

# Train Word2Vec model
w2v_model = Word2Vec(sentences=tokenized_sentences, vector_size=100, window=5, min_count=1, workers=4)

# Function to average word vectors for a document
def document_vector(doc):
    # Remove out-of-vocabulary words
    doc = [word for word in doc if word in w2v_model.wv.index_to_key]
    if not doc:
        return np.zeros(100)
    return np.mean(w2v_model.wv[doc], axis=0)

X_word2vec = np.array([document_vector(doc) for doc in tokenized_sentences])
print(f"Word2Vec Shape: {X_word2vec.shape}")

## 5. Prepare Labels

In [ ]:
# Convert labels to numeric
y = df['label'].map({'ham': 0, 'spam': 1})
print(f"Labels shape: {y.shape}")
print(f"Label distribution:\n{y.value_counts()}")

## 6. Model Training & Evaluation

### Helper Functions

In [ ]:
def train_evaluate_model(model, X_train, X_test, y_train, y_test, model_name):
    """Train and evaluate a single model"""
    # Train
    model.fit(X_train, y_train)
    
    # Predict
    y_pred = model.predict(X_test)
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, pos_label=1, zero_division=0)
    recall = recall_score(y_test, y_pred, pos_label=1, zero_division=0)
    f1 = f1_score(y_test, y_pred, pos_label=1, zero_division=0)
    
    # Print results
    print(f"\n--- {model_name} ---")
    print(f"Accuracy:  {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1 Score:  {f1:.4f}")
    
    return {
        'model': model,
        'predictions': y_pred,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1
    }

def evaluate_feature_set(X, y, feature_name, use_naive_bayes=True):
    """Evaluate all models on a feature set"""
    print(f"\n{'='*70}")
    print(f"Evaluating Models on {feature_name} Features")
    print(f"{'='*70}")
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    
    results = {}
    
    # Naive Bayes (only for non-negative features)
    if use_naive_bayes:
        nb_model = MultinomialNB()
        results['Naive Bayes'] = train_evaluate_model(nb_model, X_train, X_test, y_train, y_test, 'Naive Bayes')
    
    # SVM
    svm_model = SVC(kernel='linear', probability=True)
    results['SVM'] = train_evaluate_model(svm_model, X_train, X_test, y_train, y_test, 'SVM')
    
    # Random Forest
    rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
    results['Random Forest'] = train_evaluate_model(rf_model, X_train, X_test, y_train, y_test, 'Random Forest')
    
    return results, X_test, y_test

### Evaluate Binary Encoding

In [ ]:
binary_results, X_test_binary, y_test_binary = evaluate_feature_set(X_binary, y, "Binary Encoding")

### Evaluate Count Vectorization

In [ ]:
count_results, X_test_count, y_test_count = evaluate_feature_set(X_count, y, "Count Vectorization")

### Evaluate TF-IDF

In [ ]:
tfidf_results, X_test_tfidf, y_test_tfidf = evaluate_feature_set(X_tfidf, y, "TF-IDF")

### Evaluate Word2Vec (Word-level)

In [ ]:
# Word2Vec has negative values, so Naive Bayes won't work
w2v_results, X_test_w2v, y_test_w2v = evaluate_feature_set(X_word2vec, y, "Word2Vec (Word-level)", use_naive_bayes=False)

## 7. Find Best Model

In [ ]:
# Collect all results
all_results = {
    'Binary Encoding': binary_results,
    'Count Vectorization': count_results,
    'TF-IDF': tfidf_results,
    'Word2Vec': w2v_results
}

# Find best model
best_f1 = 0
best_model_info = None

for feature_name, models in all_results.items():
    for model_name, result in models.items():
        f1 = result['f1_score']
        if f1 > best_f1:
            best_f1 = f1
            best_model_info = (feature_name, model_name, result)

if best_model_info:
    feature_name, model_name, result = best_model_info
    print(f"\n{'='*70}")
    print(f"BEST MODEL: {model_name} with {feature_name} features")
    print(f"F1 Score: {best_f1:.4f}")
    print(f"Accuracy: {result['accuracy']:.4f}")
    print(f"Precision: {result['precision']:.4f}")
    print(f"Recall: {result['recall']:.4f}")
    print(f"{'='*70}")

## 8. Save Best Model

In [ ]:
# Create models directory
if not os.path.exists('models'):
    os.makedirs('models')

# Save best model
if best_model_info:
    feature_name, model_name, result = best_model_info
    model_path = os.path.join('models', 'best_spam_model.pkl')
    joblib.dump(result['model'], model_path)
    print(f"\nBest model saved to {model_path}")
    
    # Save appropriate vectorizer
    if 'Binary' in feature_name:
        vectorizer_path = os.path.join('models', 'binary_vectorizer.pkl')
        joblib.dump(binary_vectorizer, vectorizer_path)
    elif 'Count' in feature_name:
        vectorizer_path = os.path.join('models', 'count_vectorizer.pkl')
        joblib.dump(count_vectorizer, vectorizer_path)
    elif 'TF-IDF' in feature_name:
        vectorizer_path = os.path.join('models', 'tfidf_vectorizer.pkl')
        joblib.dump(tfidf_vectorizer, vectorizer_path)
    
    print(f"Vectorizer saved to {vectorizer_path}")

## 9. Visualize Confusion Matrix for Best Model

In [ ]:
if best_model_info:
    feature_name, model_name, result = best_model_info
    
    # Get appropriate test data
    if 'Binary' in feature_name:
        y_test_best = y_test_binary
    elif 'Count' in feature_name:
        y_test_best = y_test_count
    elif 'TF-IDF' in feature_name:
        y_test_best = y_test_tfidf
    else:
        y_test_best = y_test_w2v
    
    # Plot confusion matrix
    cm = confusion_matrix(y_test_best, result['predictions'])
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Ham', 'Spam'],
                yticklabels=['Ham', 'Spam'])
    plt.title(f'Confusion Matrix - {model_name} ({feature_name})')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()
    
    print(f"\nConfusion Matrix:\n{cm}")